# Problem Statement
#We learnt to build our own HMM-based POS tagger and implement the Viterbi algorithm using the Penn Treebank training
#corpus. The vanilla Viterbi algorithm we had written had resulted in ~87% accuracy. The approx. 13% loss of accuracy 
#was majorly due to the fact that when the algorithm encountered an unknown word(i.e. not present in the training set, such as #'Twitter'), it assigned an incorrect tag arbitrarily.#This is because, for unknown words, the emission probabilities for all candidate tags are 0, so the algorithm arbitrarily#chooses (the first) tag.In this assignment, we need to modify the Viterbi algorithm to solve the problem of unknown words using at least two techniques. Though there could be multiple ways to solve this problem,following hints can be considered:
#Find tag class that most unknown words belong to.Then identify rules (e.g. based on morphological cues) that can be used to tag unknown words?
#Viterbi algorithm chooses a random tag on encountering an unknown word.

# Approach adopted for the solution

#1.Import the libraries and the data(pre read)
#2.Understand the data.Perform EDA
#3.Split the data into train(95%) and test(5%) sets
#4.Peform vanilla Viterbi Hueristics and capture accuracy and unknown words
#5.Modify Viterbi function to perform Unigram and Combination taggers with backoff fucntionality
#6. Capture accuracy improvements with new changes
#7.Check performance of test file against vanilla Viterbi and Combination taggers to capture tagging improvements


## Data Preparation

In [364]:
#Importing required libraries
import nltk
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize


In [365]:
#reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [366]:
# looking at the first 10 tagged sentences
print(nltk_data[:10])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [367]:
# Converting the list of sentences to a list of (word, pos tag) tuples
tagged_words = [tup for sent in nltk_data for tup in sent]
print(len(tagged_words))
tagged_words[:10]

100676


[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET')]

In [368]:
# Use the set() function to list the of all tags.We then find the the number of unique POS tags in the corpus
tags = [pair[1] for pair in tagged_words]
unique_tags = set(tags)
len(unique_tags)

12

In [369]:
# Finding the most frequent tag in the corpus using the Counter() class from collections module
from collections import Counter
tag_counts = Counter(tags)
tag_counts

Counter({'NOUN': 28867,
         '.': 11715,
         'NUM': 3546,
         'ADJ': 6397,
         'VERB': 13564,
         'DET': 8725,
         'ADP': 9857,
         'CONJ': 2265,
         'X': 6613,
         'ADV': 3171,
         'PRT': 3219,
         'PRON': 2737})

In [370]:
# Listing the 5 most common tags using the most_common() method of Counter
tag_counts.most_common(5)

[('NOUN', 28867), ('VERB', 13564), ('.', 11715), ('ADP', 9857), ('DET', 8725)]

# Splitting the datase into train and test sets 

In [371]:
#Splitting the corpus into train and test,checking the lenght of each set 
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))


3718
196


In [372]:
#Printing the sentences in train set
print(test_set[:10])

[[('Volatility', 'NOUN'), ('surrounding', 'VERB'), ('his', 'PRON'), ('trades', 'NOUN'), ('occurs', 'VERB'), ('not', 'ADV'), ('because', 'ADP'), ('of', 'ADP'), ('index', 'NOUN'), ('arbitrage', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('because', 'ADP'), ('his', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('large', 'ADJ'), ('addition', 'NOUN'), ('or', 'CONJ'), ('subtraction', 'NOUN'), ('to', 'PRT'), ('a', 'DET'), ('widget', 'NOUN'), ('market', 'NOUN'), ('with', 'ADP'), ('finite', 'ADJ'), ('liquidity', 'NOUN'), ('.', '.')], [('The', 'DET'), ('British', 'NOUN'), ('Department', 'NOUN'), ('of', 'ADP'), ('Trade', 'NOUN'), ('and', 'CONJ'), ('Industry', 'NOUN'), ('ordered', 'VERB'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('competitive', 'ADJ'), ('impact', 'NOUN'), ('of', 'ADP'), ('Michelin', 'NOUN'), ('Tyre', 'NOUN'), ('PLC', 'NOUN'), ("'s", 'PRT'), ('planned', 'VERB'), ('acquisition', 'NOUN'), ('of', 'ADP'), ('National', 'NOUN'), ('Tyre', 'NOUN'), ('Service', 'NOUN'),

In [373]:
# Get the list of tagged words from train set
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95999

In [374]:
# Print the list of tagged words from train set.
train_tagged_words

[('Middlesex', 'NOUN'),
 ('Water', 'NOUN'),
 ('Co.', 'NOUN'),
 (',', '.'),
 ('offering', 'VERB'),
 ('of', 'ADP'),
 ('150,000', 'NUM'),
 ('shares', 'NOUN'),
 ('of', 'ADP'),
 ('common', 'ADJ'),
 ('stock', 'NOUN'),
 (',', '.'),
 ('via', 'ADP'),
 ('Legg', 'NOUN'),
 ('Mason', 'NOUN'),
 ('Wood', 'NOUN'),
 ('Walker', 'NOUN'),
 ('Inc.', 'NOUN'),
 ('and', 'CONJ'),
 ('Howard', 'NOUN'),
 (',', '.'),
 ('Weil', 'NOUN'),
 (',', '.'),
 ('Labouisse', 'NOUN'),
 (',', '.'),
 ('Friedrichs', 'NOUN'),
 ('Inc', 'NOUN'),
 ('.', '.'),
 ('AMR', 'NOUN'),
 ('climbed', 'VERB'),
 ('1', 'NUM'),
 ('3\\/4', 'NUM'),
 ('to', 'PRT'),
 ('73', 'NUM'),
 ('1\\/8', 'NUM'),
 ('amid', 'ADP'),
 ('rumors', 'NOUN'),
 ('that', 'ADP'),
 ('New', 'NOUN'),
 ('York', 'NOUN'),
 ('developer', 'NOUN'),
 ('Donald', 'NOUN'),
 ('Trump', 'NOUN'),
 ('was', 'VERB'),
 ('seeking', 'VERB'),
 ('financing', 'NOUN'),
 ('0', 'X'),
 ('*', 'X'),
 ('to', 'PRT'),
 ('mount', 'VERB'),
 ('a', 'DET'),
 ('new', 'ADJ'),
 (',', '.'),
 ('lower', 'ADJ'),
 ('offer'

In [375]:
# Getting tokens for tagged words from train set 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Middlesex',
 'Water',
 'Co.',
 ',',
 'offering',
 'of',
 '150,000',
 'shares',
 'of',
 'common']

In [376]:
# printing the length ofvocabulary
V = set(tokens)
print(len(V))

12068


In [377]:
# Printing the count of number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [378]:
# printing the 12 tags that are their in the train set
print(T)

{'PRON', 'NUM', 'X', 'DET', 'CONJ', 'ADP', 'ADV', 'VERB', '.', 'NOUN', 'PRT', 'ADJ'}


# Build the vanilla Viterbi based POS tagger

In [379]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

### Emission Probability

In [380]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [381]:
# Test the above function with few examples
# forthcoming
print("\n", "forthcoming")
print(word_given_tag('forthcoming', 'ADJ'))
print(word_given_tag('forthcoming', 'VERB'))
print(word_given_tag('forthcoming', 'NOUN'), "\n")
# in
print("\n", "in")
print(word_given_tag('in', 'ADP'))
print(word_given_tag('in', 'ADJ'))
print(word_given_tag('in', 'VERB'))
# Satellite
print("\n", "Satellite")
print(word_given_tag('book', 'NOUN'))
print(word_given_tag('book', 'VERB'))


 forthcoming
(1, 6127)
(0, 12956)
(0, 27480) 


 in
(1494, 9427)
(0, 6127)
(0, 12956)

 Satellite
(7, 27480)
(1, 12956)


### Transition Probability

In [382]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [383]:
# Test the above function with few examples
print(t2_given_t1(t2='NOUN', t1='ADJ'))
print(t2_given_t1('NOUN', 'DET'))
print(t2_given_t1('NOUN', 'VERB'))
print(t2_given_t1('.', 'NOUN'))


(4285, 6127)
(5312, 8332)
(1419, 12956)
(6587, 27480)


In [384]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        count_t2_t1, count_t1 = t2_given_t1(t2, t1)
        tags_matrix[i, j] = count_t2_t1/count_t1

In [385]:
# matrix in a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,PRON,NUM,X,DET,CONJ,ADP,ADV,VERB,.,NOUN,PRT,ADJ
PRON,0.007695,0.007311,0.093882,0.010004,0.005387,0.021932,0.034244,0.482493,0.040015,0.208927,0.012697,0.075414
NUM,0.001784,0.184007,0.209572,0.003270,0.013377,0.035969,0.002973,0.018728,0.118312,0.351665,0.026754,0.033591
X,0.055134,0.002844,0.075671,0.054028,0.010427,0.146130,0.026224,0.203949,0.161453,0.062243,0.184676,0.017220
DET,0.003721,0.021964,0.045247,0.005641,0.000480,0.009241,0.012602,0.040326,0.017283,0.637542,0.000240,0.205713
CONJ,0.059232,0.041647,0.008792,0.121240,0.000463,0.054604,0.055530,0.156409,0.033781,0.346599,0.005090,0.116613
ADP,0.068845,0.062692,0.034369,0.324175,0.000743,0.016866,0.013578,0.007638,0.039673,0.322372,0.001485,0.107563
ADV,0.014812,0.032258,0.023700,0.068137,0.006912,0.116853,0.078670,0.343976,0.138249,0.030612,0.014812,0.131007
VERB,0.034964,0.022769,0.218972,0.133374,0.005403,0.091386,0.082356,0.170191,0.034656,0.109525,0.031723,0.064680
.,0.065432,0.080892,0.027234,0.174636,0.058691,0.091587,0.052490,0.089520,0.092666,0.220834,0.002337,0.043592
NOUN,0.004767,0.009243,0.029258,0.013355,0.042722,0.177220,0.017140,0.147234,0.239702,0.263974,0.043122,0.012263


In [386]:
tags_df.loc['NOUN', :]

PRON    0.004767
NUM     0.009243
X       0.029258
DET     0.013355
CONJ    0.042722
ADP     0.177220
ADV     0.017140
VERB    0.147234
.       0.239702
NOUN    0.263974
PRT     0.043122
ADJ     0.012263
Name: NOUN, dtype: float32

# Viterbi Algorithm

In [387]:
len(train_tagged_words)

95999

In [388]:
# Viterbi Heuristic
def Viterbi(words,sentence,train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [389]:
#Evaluating on Test Set

random.seed(1234)

# choose random 5 sents
##rndom = [random.randint(1,196) for x in range(5)]
rndom = list(range(0,196))
# list of sents
test_run = [test_set[i] for i in rndom]
##test_run = [test_set]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_tagged_words



['Volatility',
 'surrounding',
 'his',
 'trades',
 'occurs',
 'not',
 'because',
 'of',
 'index',
 'arbitrage',
 ',',
 'but',
 'because',
 'his',
 'is',
 'a',
 'large',
 'addition',
 'or',
 'subtraction',
 'to',
 'a',
 'widget',
 'market',
 'with',
 'finite',
 'liquidity',
 '.',
 'The',
 'British',
 'Department',
 'of',
 'Trade',
 'and',
 'Industry',
 'ordered',
 'an',
 'investigation',
 'of',
 'the',
 'competitive',
 'impact',
 'of',
 'Michelin',
 'Tyre',
 'PLC',
 "'s",
 'planned',
 'acquisition',
 'of',
 'National',
 'Tyre',
 'Service',
 'Ltd',
 '.',
 'Guarantee',
 'by',
 'Dai-Ichi',
 'Kangyo',
 'Bank',
 'Ltd',
 '.',
 'Sea',
 'Containers',
 'Ltd.',
 'said',
 '0',
 'it',
 'might',
 'increase',
 'the',
 'price',
 'of',
 'its',
 '$',
 '70-a-share',
 '*U*',
 'buy-back',
 'plan',
 'if',
 '*-2',
 'pressed',
 '*-1',
 'by',
 'Temple',
 'Holdings',
 'Ltd.',
 ',',
 'which',
 '*T*-156',
 'made',
 'an',
 'earlier',
 'tender',
 'offer',
 'for',
 'Sea',
 'Containers',
 '.',
 'But',
 '``',
 'the',


### Tagging the test sentence using vanilla Viterbi

In [390]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words,train_set)
end = time.time()
difference = end-start

In [391]:
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  279.4725477695465
[('Volatility', 'NOUN'), ('surrounding', 'VERB'), ('his', 'PRON'), ('trades', 'NOUN'), ('occurs', 'VERB'), ('not', 'ADV'), ('because', 'ADP'), ('of', 'ADP'), ('index', 'NOUN'), ('arbitrage', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('because', 'ADP'), ('his', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('large', 'ADJ'), ('addition', 'NOUN'), ('or', 'CONJ'), ('subtraction', 'PRON'), ('to', 'PRT'), ('a', 'DET'), ('widget', 'NOUN'), ('market', 'NOUN'), ('with', 'ADP'), ('finite', 'PRON'), ('liquidity', 'NOUN'), ('.', '.'), ('The', 'DET'), ('British', 'ADJ'), ('Department', 'NOUN'), ('of', 'ADP'), ('Trade', 'NOUN'), ('and', 'CONJ'), ('Industry', 'NOUN'), ('ordered', 'VERB'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('competitive', 'ADJ'), ('impact', 'NOUN'), ('of', 'ADP'), ('Michelin', 'NOUN'), ('Tyre', 'NOUN'), ('PLC', 'NOUN'), ("'s", 'PRT'), ('planned', 'VERB'), ('acquisition', 'NOUN'), ('of', 'ADP'), ('National', 'NOUN'),

### Vanilla Viterbi Accuracy

In [392]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('The accuracy of vanilla vitrebi is :', accuracy)

The accuracy of vanilla vitrebi is : 0.8960872354073124


### Incorrectly Tagged Words - Unknown Words

In [393]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('or', 'CONJ'), (('subtraction', 'PRON'), ('subtraction', 'NOUN'))],
 [('with', 'ADP'), (('finite', 'PRON'), ('finite', 'ADJ'))],
 [('The', 'DET'), (('British', 'ADJ'), ('British', 'NOUN'))],
 [('.', '.'), (('Guarantee', 'PRON'), ('Guarantee', 'NOUN'))],
 [('$', '.'), (('70-a-share', 'PRON'), ('70-a-share', 'ADJ'))],
 [('*U*', 'X'), (('buy-back', 'NOUN'), ('buy-back', 'ADJ'))],
 [('which', 'DET'), (('*T*-156', 'PRON'), ('*T*-156', 'X'))],
 [('earlier', 'ADJ'), (('tender', 'NOUN'), ('tender', 'ADJ'))],
 [('is', 'VERB'), (('workable', 'PRON'), ('workable', 'ADJ'))],
 [('High', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('came', 'VERB'), (('out', 'PRT'), ('out', 'ADV'))],
 [('a', 'DET'), (('publication', 'PRON'), ('publication', 'NOUN'))],
 [('at', 'ADP'), (('102', 'PRON'), ('102', 'NUM'))],
 [('with', 'ADP'), (('102', 'PRON'), ('102', 'NUM'))],
 [('102', 'NUM'), (('12\\/32', 'PRON'), ('12\\/32', 'NUM'))],
 [('Pat', 'NOUN'), (("D'Amico", 'PRON'), ("D'Amico", 'NOUN'))],
 [('Baltim

In [394]:
len(incorrect_tagged_cases)

486

### Tokenizing the sentences in the test file

In [395]:
## Preparing data for the test file and its sentences
sentence_test ='Android is a mobile operating system developed by Google.Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.Google and Twitter made a deal in 2015 that gave Google access to Twitter firehose.Twitter is an online news and social networking service on which users post and interact with messages known as tweets.Before entering politics, Donald Trump was a domineering businessman and a television personality.The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.Show me the cheapest round trips from Dallas to Atlanta.I would like to see flights from Denver to Philadelphia.Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.NASA invited social media users to experience the launch of ICESAT-2 Satellite.'
sentence_test_split = sentence_test.split()
words = word_tokenize(sentence_test)

# Statistics of vanilla Viterbi algorithm
#### 486 words have been incorrectly tagged and are unknown words
#### Accuracy is 89.6%

# Solve the problem of unknown words

# Now we will use two techniques to modify the vanilla Viterbi algorithm - First by using Unigram Tagger and then by using the combination of Unigram,Bigram and Rule Based(Morphing) Taggers.The functions created are unigram() and morph() which is called from inside the Vitrebi function and return accuracies of Viterbi,Unigram and morph functions. 

### unigram Function - returns accuracy of tagging for the same input as vanilla Viterbi 

In [396]:
def unigram(data):
    unigram_tagger = nltk.UnigramTagger(train_set)
    accuracy = unigram_tagger.evaluate(data)
    return accuracy

### morph function - returns accuracy of tagging for the same input as vanilla Viterbi.Also returns the tags for thesentences given in the testing file 

In [397]:
def morph(data_input,test_sent):

    patterns = [
        (r'.*ing$', 'VERB'),
        (r'(The|the|A|a|An|an|some|Some|most|Most|every|Every|no|No)$', 'DET'),
        (r'(To|to|In|in)$', 'ADP'),# gerund
        (r'(I|i|you|You|he|He|she|She|it|It|we|We|they|They)$', 'PRON'),
        (r'(and|And|or|Or|but|But)$', 'CONJ'),
        (r'.*ed$', 'VERB'),               # past tense
        (r'.*es$', 'VERB'),
        (r'.*able$', 'ADJ'),
        (r'.*ness$', 'NOUN'),
        (r'.*s$', 'NOUN'),  
        (r'.*ly$', 'ADV'),# 3rd singular present
        (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
        (r'^[A-Z]','NOUN'),
        (r'.*', 'NOUN')                    # nouns
    
         ]
#Rule-Based (Regular Expression) Tagger

    rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
    lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

# Bigram backed up by the lexicon tagger
    bigram_tagger = nltk.BigramTagger(train_set, backoff=lexicon_tagger)

    accuracy = bigram_tagger.evaluate(data_input)
    tag = bigram_tagger.tag(test_sent)

    return (accuracy,tag)

### Modification to original vanilla Viterbi funtion by implementing two techniques to deal with unknown words.

In [398]:
# Modified Viterbi Heuristic
def Viterbi_Mod(words,sentence,train_bag = train_tagged_words):
    state = []
    #unigram_tagger = nltk.UnigramTagger(train_set)

    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        tagged = list(zip(words, state))
        check = [i for i, j in zip(tagged, test_run_base) if i == j] 
        accuracy = len(check)/len(tagged)
    
               
# Call the unigram function to check accuracy of tagged words - Technique 1  
    unigram_accuracy = unigram(sentence)

# Call the morph function to check accuracy of tagged words - Technique 2         
    morph_data = morph(sentence,sentence_test_split)
    morph_list = [morph_data]

    #morph accuracy
    morph_select_accuracy = [i[0] for i in morph_list]
    morph_accuracy = morph_select_accuracy[0]
    
    #morph Tag
    morph_select_tag = [i[1] for i in morph_list]
    morph_tagging = morph_select_tag[0]           
    
    return (accuracy,unigram_accuracy,morph_accuracy,morph_tagging)



In [399]:
# tagging the test sentences
start = time.time()
tagged_mod_seq = Viterbi_Mod(test_tagged_words,test_set)
end = time.time()
difference = end-start

### Comparing the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [400]:
print('The accuracy for vanilla Viterbi is :',tagged_mod_seq[0])
print('The accuracy for unigram tagger is :',tagged_mod_seq[1])
print('The accuracy for combination tagger is :',tagged_mod_seq[2])


The accuracy for vanilla Viterbi is : 0.8960872354073124
The accuracy for unigram tagger is : 0.8945905494975411
The accuracy for combination tagger is : 0.9506093649775497


### The modification technique 2 - backoff using bigram,unigram and rule based perfomed with better accuracy (95%) compared to unigram technique 1 (89.4%).Unigram had the same accuracy as that of Viterbi

# We now see how the vanilla Viterbi and modification perform on the given test file given.

### Applying vanilla Viterbi to the test sentences.

In [401]:
tagged_test_seq = Viterbi(words,words)
print(tagged_test_seq)

[('Android', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google.Android', 'PRON'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'PRON'), ('worldwide', 'PRON'), ('on', 'ADP'), ('smartphones', 'PRON'), ('since', 'ADP'), ('2011', 'PRON'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013.Google', 'PRON'), ('and', 'CONJ'), ('Twitter', 'PRON'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'PRON'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'PRON'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'PRON'), ('firehose.Twitter', 'PRON'), ('is', 'VERB'), ('an', 'DET'), ('online', 'PRON'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', 'CONJ'), ('interact', 'PRON'), ('with', 'AD

In [402]:
len(tagged_test_seq)

161

### Picking 3 elements that have been incorrectly tagged by vanilla Viterbi

In [403]:
from operator import itemgetter
itemgetter(0,158,159)(tagged_test_seq)


(('Android', 'PRON'), ('ICESAT-2', 'PRON'), ('Satellite', 'PRON'))

In [404]:
len(tagged_mod_seq[3])

158

### The correct tagging of the three elements after applying the modification technique 2 - Combined tagger (unigram,bigram and rule based)

In [405]:
from operator import itemgetter
itemgetter(0,156,157)(tagged_mod_seq[3])

(('Android', 'NOUN'), ('ICESAT-2', 'NOUN'), ('Satellite.', 'NOUN'))

### Comparing the three elements from test sentence between Viterbi and Modified tagger

In [406]:
from operator import itemgetter
print('Incorrectly tagged by Viterbi :',itemgetter(0,158,159)(tagged_test_seq))
print('\n')
print('Correctly tagged by the Modified Combined Tagger :',itemgetter(0,156,157)(tagged_mod_seq[3]))

Incorrectly tagged by Viterbi : (('Android', 'PRON'), ('ICESAT-2', 'PRON'), ('Satellite', 'PRON'))


Correctly tagged by the Modified Combined Tagger : (('Android', 'NOUN'), ('ICESAT-2', 'NOUN'), ('Satellite.', 'NOUN'))
